In [1]:
from set_math import *
from datasets_init import *

sympy imported
pandas imported
numpy imported
scipy imported


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


statsmodels imported
statsmodels.formula imported
statasmodels.stats.api imported
train.csv good
test.csv good
you may now begin


# Preprocessing

### np.nan value

In [2]:
train.shape

(647054, 7)

In [3]:
train.dropna(how='any', inplace=True)
train.shape

(642925, 7)

#### Weekday

In [4]:
# 요일 numerical encoding

wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

#### Scancount

In [5]:
def purchase_return(train):
    """
    add new column return
    """
    train.loc[train.ScanCount < 0, "Return"] = 1
    train.loc[train.Return != 1, "Return"] = 0
    train.loc[train.Return == 1, "ScanCount"] = 0

In [6]:
train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])
purchase_return(train)


### Drop `Upc` , `HEALTH AND BEAUTY AIDS`

In [7]:
train.drop("Upc", axis=1, inplace=True)

In [8]:
train = train[train['DepartmentDescription'] != 'HEALTH AND BEAUTY AIDS']

In [9]:
print(train.shape)
print(train.columns)
train.head()

(642923, 7)
Index(['TripType', 'VisitNumber', 'Weekday', 'ScanCount',
       'DepartmentDescription', 'FinelineNumber', 'Return'],
      dtype='object')


,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,Return
0,999,5,4,0,FINANCIAL SERVICES,1000.0,1.0
1,30,7,4,1,SHOES,8931.0,0.0
2,30,7,4,1,PERSONAL CARE,4504.0,0.0
3,26,8,4,2,PAINT AND ACCESSORIES,3565.0,0.0
4,26,8,4,2,PAINT AND ACCESSORIES,1017.0,0.0


### 학습을 위한 전처리 준비

In [10]:
def count_per_DD(data):
    """
    OneHotEncoding DepartmentDescription 
    then multiply ScanCount.
    """
    dummies = pd.get_dummies(data.DepartmentDescription)
    data_dummies = dummies.apply(lambda x: x * data["ScanCount"])
    data_dummies = data_dummies.replace(-0, 0)
    data[dummies.columns] = data_dummies

In [11]:
count_per_DD(train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [12]:
train_max_groupby = train[["TripType", "VisitNumber", "Weekday", "Return"]] # max로 groupby 하는 피처들
group_max = train_max_groupby.groupby("VisitNumber").agg(np.max).reset_index()

In [13]:
dpts = train.DepartmentDescription.unique()

In [14]:
train_dpts = train.loc[:, "1-HR PHOTO":"WIRELESS"]
train_scvn = train[["VisitNumber", "ScanCount"]]
train_sum_groupby = pd.concat([train_scvn, train_dpts], axis=1)
print(train_sum_groupby.shape)
train_sum_groupby.head()

(642923, 69)


,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
group_sum = train_sum_groupby.groupby("VisitNumber").agg(np.sum).reset_index()
print(group_sum.shape)
group_sum.head()

(94247, 69)


,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,2,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8,29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
triptype_df = train[['TripType', 'VisitNumber']].groupby(['VisitNumber']).mean()
triptype_df.reset_index(drop=False, inplace=True)
triptype_df.shape

(94247, 2)

In [17]:
return_df = train[['Return', 'VisitNumber']].groupby(['VisitNumber']).sum()
return_df['Return'] = return_df['Return'].apply(lambda x: 1 if x > 0 else 0)
return_df.reset_index(drop=False, inplace=True)
return_df.shape

(94247, 2)

In [18]:
weekday_df = train[['Weekday', 'VisitNumber']].groupby(['VisitNumber']).mean()
weekday_df.reset_index(drop=False, inplace=True)
weekday_df.shape

(94247, 2)

In [19]:
group_sum['TripType'] = triptype_df['TripType']
group_sum['Return'] = return_df['Return']
group_sum['Weekday'] = weekday_df['Weekday']

In [20]:
# from sklearn.preprocessing import StandardScaler

# #scaling refund, purchase 
# scaler = StandardScaler()
# group_sum['ScanCount'] = scaler.fit_transform(group_sum['ScanCount'].values.reshape(-1, 1))

In [21]:
train_df = group_sum.copy()

In [22]:
print(train_df.shape)
train_df.head()

(94247, 72)


,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,TripType,Return,Weekday
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,999,1,4
1,7,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,30,0,4
2,8,29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,26,1,4
3,9,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,4
4,10,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,4


In [23]:
X_df = train_df.drop(['TripType', 'VisitNumber'], axis=1)
X_df.shape

(94247, 70)

In [24]:
y_df = train_df[['TripType']]
y_df.shape

(94247, 1)

### Train, Test split

In [25]:
from sklearn.model_selection import train_test_split

X, y = X_df, y_df
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [33]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y_train)
lb.classes_

array([  3,   4,   5,   6,   7,   8,   9,  12,  14,  15,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44, 999])

In [34]:
y_train_bi = lb.transform(y)
y_train_bi

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
lb.inverse_transform(y_train_bi)

array([999,  30,  26, ...,  39,  39,   8])

In [39]:
y_train_bi.shape

(94247, 38)

# Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

### C = 1.0

In [47]:
LR = LogisticRegression(C=1.0).fit(X_train, y_train)
LR.predict(X_test)
print("Train score: {:.3f}".format(LR.score(X_train, y_train)))
print("Test score: {:.3f}".format(LR.score(X_test, y_test)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train score: 0.643
Test score: 0.629


# C = 10

가장 높게 나온 ... 

In [50]:
LR10 = LogisticRegression(C=10).fit(X_train, y_train)
LR10.predict(X_test)
print("Train score: {:.3f}".format(LR10.score(X_train, y_train)))
print("Test score: {:.3f}".format(LR10.score(X_test, y_test)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train score: 0.644
Test score: 0.629


### C = 100

In [49]:
LR = LogisticRegression(C=100).fit(X_train, y_train)
LR.predict(X_test)
print("Train score: {:.3f}".format(LR.score(X_train, y_train)))
print("Test score: {:.3f}".format(LR.score(X_test, y_test)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train score: 0.645
Test score: 0.628


In [52]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [53]:
print("LR(C=10) confusion matrix : \n",
      confusion_matrix(y_train, LR10.predict(X_train)), "\n")

print("LR(C=10) classification report : \n",
      classification_report(y_train, LR10.predict(X_train)))

LR(C=10) confusion matrix : 
 [[2582    0   10 ...,    0    0   26]
 [   0    0  110 ...,    0    0    2]
 [   1    0 1407 ...,    0    0   27]
 ..., 
 [   2    0   16 ...,    0    0    5]
 [   0    0    6 ...,    0   45    1]
 [ 527    0   40 ...,    0    0 4623]] 

LR(C=10) classification report : 
              precision    recall  f1-score   support

          3       0.79      0.93      0.85      2771
          4       0.00      0.00      0.00       255
          5       0.62      0.55      0.59      2540
          6       0.70      0.47      0.56       946
          7       0.67      0.66      0.66      4341
          8       0.63      0.88      0.73      9037
          9       0.65      0.78      0.71      7135
         12       0.32      0.03      0.06       199
         14       0.00      0.00      0.00         4
         15       0.58      0.34      0.43       731
         18       0.52      0.39      0.45       421
         19       0.50      0.12      0.20       279
       

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
